In [1]:
import rospy
import moveit_commander
import baxter_interface
import moveit_msgs.msg
import geometry_msgs.msg
import numpy as np
import time
import pickle
import os.path as path
from tqdm import tqdm_notebook as tqdmn

In [2]:
# defining baxter specific variables

JOINT_LIMITS = {
    's0': {
        'min': 0.00,
        'max': 1.3,
    },
    's1': {
        'min': -2.0,
        'max': 0.9,
    },
    'e0': {
        'min':  -3.05,
        'max': 3.05,
    },
    'e1': {
        'min': 0.0,
        'max': 2.5,
    },
    'w0': {
        'min': -3.05,
        'max': 3.05,
    },
    'w1': {
        'min': -1.4,
        'max': 1.9,
    },
    'w2': {
        'min': -3.05,
        'max': 3.05,
    },
}
joint_names = ['s0', 's1', 'e0', 'e1', 'w0', 'w1', 'w2']

In [3]:
moveit_commander.roscpp_initialize('test')
rospy.init_node('test')

In [4]:
# initialize moveit scene
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group = moveit_commander.MoveGroupCommander("right_arm")

In [5]:
# set goal tolerances
group.set_goal_joint_tolerance(0.001)
group.set_max_velocity_scaling_factor(0.5)

In [6]:
# in a loop choose 10 random points within limits, generate collision free plans, try and catch for validity of goal
file_seed = path.expanduser('~/data/moveit_data/bend_dof_')
for file_iter in tqdmn(xrange(2), desc='File Count'):
    joint_goal = {}
    plans = []
    filename = file_seed + str(file_iter) + '.pkl'
    waypoint_success = 0
    while waypoint_success < 10:
        for joint in range(len(joint_names)-1):
            if joint_names[joint] == 's1' or joint_names[joint] == 'e1' or joint_names[joint] == 'w1':
                joint_goal['right_' + joint_names[joint]] = np.random.uniform(JOINT_LIMITS[joint_names[joint]]['min'], 
                                                                        JOINT_LIMITS[joint_names[joint]]['max'])
            else:
                joint_goal['right_' + joint_names[joint]] = 0.0
        group.clear_pose_targets()
        group.set_joint_value_target(joint_goal)
        plan = group.plan()
        if waypoint_success == 0:
            plans.append(plan.joint_trajectory.joint_names)
        for point in plan.joint_trajectory.points:
            plans.append(point.positions)
        if (len(plan.joint_trajectory.points)>0):
            start_time = time.time()
            group.go(wait=True)
#             print "Calculated Plan was of length: " + str(len(plan.joint_trajectory.points))
#             print "Total execution time: " + str(time.time()-start_time) + " seconds"
            waypoint_success += 1
#         else:
#             print "No Plan found for Goal:"
#             print joint_goal
#         print ""
    with open(filename, 'w') as f:
        pickle.dump(plans, f)
# TODO: look at -
# http://wiki.ros.org/industrial_trajectory_filters, http://wiki.ros.org/industrial_trajectory_filters/Tutorials/filters_inside_moveit
# https://groups.google.com/forum/#!topic/moveit-users/CCj9Tam2jiw